In [ ]:
import matplotlib
%matplotlib widget

import numpy as np
from split_dataset import SplitDataset
from pathlib import Path
import flammkuchen as fl
from tifffile import imread
import matplotlib.pyplot as plt 
from fimpylab.core.twop_experiment import TwoPExperiment
from bouterin.plots.stimulus_log_plot import get_paint_function

In [ ]:
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\rf")
fish_list = list(master.glob("*_f*"))
fish_dir = fish_list[0] 
f = fish_dir / 'suite2p' / 'combined'

try:
    with open(next(fish_dir.glob("*metadata.json"))) as i:
        metadata = json.load(i)
    fish_id = metadata['general']['fish_id']
except:
    fish_id = ""
print(fish_dir)
print(fish_id)

In [ ]:
traces = fl.load(fish_dir / "traces.h5")['traces'][:, 0:-2]
fs = 3
t = np.arange(np.shape(traces)[1]) / fs
num_traces, len_rec = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
new_len_rec = int(len_rec/3)

In [ ]:
norm_traces = np.copy(traces)
norm_traces=norm_traces.T# need to transpose it since the functions work like that 
sd=np.nanstd(norm_traces)
mean=np.nanmean(norm_traces)
norm_traces=norm_traces-mean #numerator in the formula for z-score 
norm_traces=norm_traces/sd
traces=norm_traces.T

In [ ]:
trial_traces = np.zeros((3, num_traces, new_len_rec))
for i in range(3):
    t1 = i * new_len_rec
    t2 = t1 + new_len_rec
    trial_traces[i] = traces[:, t1:t2]
avg_traces = np.nanmean(trial_traces, 0)

In [ ]:
print(np.shape(avg_traces))

In [ ]:
exp = TwoPExperiment(path=fish_dir)
num_subtrials = 16
num_rep = 3
num_trials = num_rep * num_subtrials

stimulus_log = exp.load_session_log(log_name='stimulus_log', session_idx=0)
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0]

In [ ]:
### reorder traces:
# take the second forward trial to the end
new_traces = np.copy(avg_traces)
forward_2nd = new_traces[:,(8 * 45):(9*45)]
new_traces[:,(8 * 45):(15 * 45)] = avg_traces[:,(9 * 45):(16 * 45)]
new_traces[:,(15*45):] = forward_2nd
fig_tet, ax_tet = plt.subplots(3,1, figsize=(15,10))
ax_tet[0].imshow(avg_traces)
ax_tet[1].imshow(new_traces)
ax_tet[2].imshow(avg_traces-new_traces)
print(np.shape(new_traces))

In [ ]:
max_vals = np.zeros((num_traces, 2, 8))
mean_vals = np.zeros((num_traces, 2, 8))

for i in range(8):
    t1 = (i * 15) + 5
    t2 = t1 + (fs * 10)
    tmp_traces = avg_traces[:, t1:t2]
    tmp_max = np.amax(tmp_traces, axis=1)
    max_vals[:, 0, i] = tmp_max
    
    tmp_mean = np.nanmean(tmp_traces, axis=1)
    mean_vals[:, 0, i] = tmp_mean
    
    t1 = ((8-i) * 15) + 5
    t2 = t1 + (fs * 10)
    tmp_traces = avg_traces[:, t1:t2]
    tmp_max = np.amax(tmp_traces, axis=1)
    max_vals[:, 1, 7-i] = tmp_max
    
    tmp_mean = np.nanmean(tmp_traces, axis=1)
    mean_vals[:, 1, 7-i] = tmp_mean

In [ ]:
max_diff = max_vals[:, 1, :] - max_vals[:, 0, :]
mean_diff = mean_vals[:, 1, :] - mean_vals[:, 0, :]
fig2, ax2 = plt.subplots(2, 2, figsize=(13,5))
ax2[0,0].imshow(max_diff, aspect="auto")#, vmin=-20, vmax=1500)
ax2[1,0].imshow(mean_diff, aspect="auto")#, vmin=-20, vmax=1500)

tmp_max_val = np.amax(abs(max_diff), axis=1)
ind_sort = np.argsort(tmp_max_val)
max_diff_sort = max_diff[ind_sort]
tmp_max_val = np.amax(abs(mean_diff), axis=1)
ind_sort = np.argsort(tmp_max_val)
mean_diff_sort = mean_diff[ind_sort]

ax2[0,1].imshow(max_diff_sort, aspect="auto")#, vmin=-20, vmax=1500)
ax2[1,1].imshow(mean_diff_sort, aspect="auto")#, vmin=-20, vmax=1500)
print(max_diff)

In [ ]:
roi_map = fl.load(fish_dir / "merged_rois.h5")['stack'][0:12,:,:]
fig3, ax3 = plt.subplots(3, 5, figsize=(12, 12))
num_rois = num_traces
num_planes = np.shape(roi_map)[0]

for i in range(num_planes):
    r = i // 5
    c = np.mod(i, 5)
    
    roi_layer = roi_map[i]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax3[r, c].imshow(roi_layer, cmap="rainbow")
    
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
roi_map_max_diff = np.zeros_like(roi_map)
roi_map_max_diff_pos = np.zeros_like(roi_map)
roi_map_max_diff_neg = np.zeros_like(roi_map)

roi_map_max_diff_pos_val = np.zeros_like(roi_map)
roi_map_max_diff_neg_val = np.zeros_like(roi_map)

roi_map_max_0 = np.zeros_like(roi_map)
roi_map_max_1 = np.zeros_like(roi_map_max_0)
roi_map_max_2 = np.zeros_like(roi_map_max_0)
roi_map_max_3 = np.zeros_like(roi_map_max_0)
roi_map_max_4 = np.zeros_like(roi_map_max_0)
roi_map_max_5 = np.zeros_like(roi_map_max_0)
roi_map_max_6 = np.zeros_like(roi_map_max_0)
roi_map_max_7 = np.zeros_like(roi_map_max_0)

max_diff = mean_diff
max_diff_filtered = np.copy(max_diff)
max_diff_filtered[np.where(abs(max_diff_filtered) < 200)[0]] = 0
tmp_max = np.argmax(abs(max_diff), axis=1)
tmp_max_pos = np.argmax(max_diff, axis=1)
tmp_max_neg = np.argmax(max_diff * -1, axis=1)

print(tmp_max)
tmp_max_val = np.amax(abs(max_diff), axis=1)
tmp_max_pos_val = np.amax(max_diff, axis=1)
tmp_max_neg_val = np.amax(max_diff * -1, axis=1)

#tmp_max[np.where(tmp_max_val < 200)[0]] = -1
#tmp_max_pos[np.where(tmp_max_pos_val < 200)[0]] = -1
#tmp_max_neg[np.where(tmp_max_neg_val < 200)[0]] = -1
for i in range(0, num_rois):
    
    roi_map_max_diff[np.where(roi_map == (i + 1))] = tmp_max[i] + 1
    roi_map_max_diff_pos[np.where(roi_map == (i + 1))] = tmp_max_pos[i] + 1
    roi_map_max_diff_neg[np.where(roi_map == (i + 1))] = tmp_max_neg[i] + 1
    
    roi_map_max_diff_pos_val[np.where(roi_map == (i + 1))] = tmp_max_pos_val[i]
    roi_map_max_diff_neg_val[np.where(roi_map == (i + 1))] = tmp_max_neg_val[i]
    
    roi_map_max_0[np.where(roi_map == (i + 1))] = max_diff[i, 0]
    roi_map_max_1[np.where(roi_map == (i + 1))] = max_diff[i, 1]
    roi_map_max_2[np.where(roi_map == (i + 1))] = max_diff[i, 2]
    roi_map_max_3[np.where(roi_map == (i + 1))] = max_diff[i, 3]
    roi_map_max_4[np.where(roi_map == (i + 1))] = max_diff[i, 4]
    roi_map_max_5[np.where(roi_map == (i + 1))] = max_diff[i, 5]
    roi_map_max_6[np.where(roi_map == (i + 1))] = max_diff[i, 6]
    roi_map_max_7[np.where(roi_map == (i + 1))] = max_diff[i, 7]

roi_map_diff_merged = np.zeros((8, np.shape(roi_map)[0], np.shape(roi_map)[1], np.shape(roi_map)[2]))
roi_map_diff_merged[0] = roi_map_max_0
roi_map_diff_merged[1] = roi_map_max_1
roi_map_diff_merged[2] = roi_map_max_2
roi_map_diff_merged[3] = roi_map_max_3
roi_map_diff_merged[4] = roi_map_max_4
roi_map_diff_merged[5] = roi_map_max_5
roi_map_diff_merged[6] = roi_map_max_6
roi_map_diff_merged[7] = roi_map_max_7

In [ ]:
print(np.max(roi_map_max_6))

In [ ]:
ind_sort = np.argsort(tmp_max_val)
sorted_traces = avg_traces[ind_sort]
np.shape(sorted_traces)
fig10, ax10 = plt.subplots(1, 1, figsize=(12, 12))
ax10.imshow(sorted_traces,extent=[0,  1000, 0, 1000])
print(ind_sort)
fig10.savefig(str(fish_dir/'traces_sorted_max_diff_abs.jpg'), dpi=300)

In [ ]:
fig4, ax4 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax4[r, c].imshow(roi_layer, cmap=cm_roi)
    ax4[r, c].axis('off')
    ax4[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
#fig4.savefig(str(fish_dir/'rois_direction_of_max_abs_diff.jpg'))

In [ ]:
vmin = np.zeros(8)
vmax = np.zeros(8)
for i in range(8):
    tmp_map = roi_map_diff_merged[i]
    vmin[i] = np.min(tmp_map)
    vmax[i] = np.max(tmp_map)
print(vmin)
print(vmax)

In [ ]:
fig5, ax5 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
direction_list = ['f', 'fr', 'r', 'br', 'b', 'bl', 'l', 'fl']
title_list = ['forward', 'forward-right', 'right', 'backward-right', 'backward', 'backward-left', 'left', 'foward-left']
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for j in range(8):
    for i in range(num_planes):
        r = i // 4
        c = np.mod(i, 4)
        
        roi_layer_orig = roi_map[i]
        roi_layer = roi_map_diff_merged[j, i]
        #roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
        roi_layer = np.ma.masked_where(roi_layer_orig < 1, roi_layer)
        roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

        #ax4[r, c].imshow(roi_layer, cmap="rainbow")
        ax5[r, c].imshow(roi_layer, cmap="coolwarm",  vmin=min(vmax*-1), vmax=max(vmax))
        ax5[r, c].axis('off')
        ax5[r, c].set_title("z" + str(i))
        fig5.suptitle(title_list[j])
        #print(roi_layer)

    
    plt.show()
    file_name ='rois_mean_diff_' + direction_list[j] + '_' + fish_id + '.jpg'
    fig5.savefig(str(fish_dir/file_name), dpi=300)

In [ ]:
fig6, ax6 = plt.subplots(2, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
direction_list = ['f', 'fr', 'r', 'br', 'b', 'bl', 'l', 'fl']
title_list = ['forward', 'forward-right', 'right', 'backward-right', 'backward', 'backward-left', 'left', 'foward-left']
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for j in range(8):
    for i in range(num_planes):
        r = j // 4
        c = np.mod(j, 4)
        
        roi_layer_orig = roi_map[i]
        roi_layer = roi_map_diff_merged[j, i]
        #roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
        roi_layer = np.ma.masked_where(roi_layer_orig < 1, roi_layer)
        roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

        ax6[r, c].imshow(roi_layer, cmap="coolwarm", vmin=min(vmax*-1), vmax=max(vmax))
        ax6[r, c].axis('off')
        ax6[r, c].set_title(title_list[j])
        #print(roi_layer)

    
plt.show()
file_name ='rois_mean_diff_overlay_' + fish_id + '.jpg'
fig6.savefig(str(fish_dir/file_name), dpi=300)

In [ ]:
fig6, ax6 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_pos[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax6[r, c].imshow(roi_layer, cmap=cm_roi)
    ax6[r, c].axis('off')
    ax6[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig6.savefig(str(fish_dir/'rois_direction_of_max_diff_pos.jpg'), dpi=300)

In [ ]:
fig7, ax7 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_neg[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax7[r, c].imshow(roi_layer, cmap=cm_roi)
    ax7[r, c].axis('off')
    ax7[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig7.savefig(str(fish_dir/'rois_direction_of_max_abs_diff_neg.jpg'))

In [ ]:
fig8, ax8 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_pos_val[i]
    roi_layer[0, 1:3] = [-5, 5]
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax8[r, c].imshow(roi_layer, cmap="bwr")
    ax8[r, c].axis('off')
    ax8[r, c].set_title("z" + str(i))

    
plt.show()
fig8.savefig(str(fish_dir/'rois_max_pos_diff_value.jpg'))

In [ ]:
fig9, ax9 = plt.subplots(3, 4, figsize=(12, 12))

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_neg_val[i]
    roi_layer[0, 1:3] = [-5, 5]
    #roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax9[r, c].imshow(roi_layer, cmap="bwr")
    ax9[r, c].axis('off')
    ax9[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig9.savefig(str(fish_dir/'rois_max_neg_diff_value.jpg'))

In [ ]:
fig5, ax5 = plt.subplots(3, 5, figsize=(12, 12))

for i in range(num_planes):
    r = i // 5
    c = np.mod(i, 5)
    
    roi_layer = roi_map_max_5[i]
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax5[r, c].imshow(roi_layer, cmap="bwr")#rainbow")
    ax5[r, c].axis('off')
    #print(roi_layer)

    
plt.show()